# **Convolutional Neural Network**
**Starting parameters**:

- Number of epochs: 100
- Learning rate: 0.0005
- Layers: [64, 32, 32]
- Dropout: 0.3


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For preprocessing
import tensorflow as tf

# For modeling
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random

In [4]:
pkl_path_1D = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated Test/test_features_3s_all_1D.pkl'

# Load the pickle file
with open(pkl_path_1D, 'rb') as file:
    test_1D = pickle.load(file)
del file

In [5]:
pkl_path_2D = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated Test/test_features_3s_all_2D.pkl'

# Load the pickle file
with open(pkl_path_2D, 'rb') as file:
    test_2D = pickle.load(file)
del file

In [8]:
test_f_1D = test_1D['test'].copy()
test_f_2D = test_2D['test'].copy()

test_labels = test_f_1D['label'].copy()
temp = test_f_1D.copy()
del temp['label']
test_features_1D = temp

temp = test_f_2D.copy()
del temp['label']
test_features_2D = temp

## **Shuffling Data**

In [9]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [10]:
test_y_1D, test_X_1D = shuffle_data(test_labels, test_features_1D)

In [11]:
test_y_2D, test_X_2D = shuffle_data(test_labels, test_features_2D)

In [37]:
# pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_3s_all_2D.pkl'

# # Load the pickle file
# with open(pkl_path, 'rb') as file:
#     data = pickle.load(file)
# del file

## **CNN Model**

In [12]:
def evaluate_model(val_y, val_yhat, val_yhat_result, num_classes=20):

    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_yhat, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    val_score = {'f1': f1, 'auc': auc_score}

    return val_score

In [13]:
def tile_and_crop(feature, target_size):
    tiled = np.tile(feature, (1, target_size // feature.shape[1] + 1, 1))
    return tiled[:, :target_size, :]

In [14]:
test_results = {}

# **Traditional Models**

In [15]:
# # 1D

# rf = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/RF/RF_melspectrogram_cqt_mfcc.keras')
# xgboost = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/XGB/XGB_melspectrogram_cqt_mfcc.keras')
# svm = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/SVM/SVM_melspectrogram_cqt_mfcc.keras')
# knn = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/KNN/KNN_melspectrogram_cqt_mfcc.keras')

# rf = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/RF/RF_melspectrogram_chroma_mfcc.keras')
# xgboost = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/XGB/XGB_melspectrogram_chroma_mfcc.keras')
# svm = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/SVM/SVM_melspectrogram_chroma_mfcc.keras')
# knn = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/KNN/KNN_melspectrogram_chroma_mfcc.keras')

# **Deep Learning Models**

In [16]:
# 2D

ffnn_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/FFNN/FFNN_melspectrogram_cqt_mfcc.keras')
cnn_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/CNN/CNN_melspectrogram_cqt_mfcc.keras')
EB0_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/EfficientB0_melspectrogram_cqt_mfcc.keras')
EB4_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/EfficientB4_melspectrogram_cqt_mfcc.keras')
Res_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/ResNet50_melspectrogram_cqt_mfcc.keras')
VGG_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/VGG16_melspectrogram_cqt_mfcc.keras')

ffnn_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/FFNN/FFNN_melspectrogram_chroma_mfcc.keras')
cnn_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/CNN/CNN_melspectrogram_chroma_mfcc.keras')
EB0_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/EfficientB0_melspectrogram_chroma_mfcc.keras')
EB4_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/EfficientB4_melspectrogram_chroma_mfcc.keras')
Res_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/ResNet50_melspectrogram_chroma_mfcc.keras')
VGG_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/VGG16_melspectrogram_chroma_mfcc.keras')

# **Preparing Test Data**

## 1D Testing Stuff

In [46]:
testing_1D_melmfccchroma3D = np.concatenate((test_X_2D['mfcc'], test_X_2D['chroma'], test_X_2D['melspectrogram']), axis=1)
testing_1D_melmfccchroma = testing_1D_melmfccchroma3D.reshape(testing_1D_melmfccchroma3D.shape[0], -1)
print(testing_1D_melmfccchroma.shape)

testing_1D_melmfcccqt3D = np.concatenate((test_X_2D['mfcc'], test_X_2D['cqt'], test_X_2D['melspectrogram']), axis=1)
testing_1D_melmfcccqt = testing_1D_melmfcccqt3D.reshape(testing_1D_melmfcccqt3D.shape[0], -1)
print(testing_1D_melmfcccqt.shape)

(2426, 41440)
(2426, 60088)


In [47]:
test_y_2D.shape

(2426,)

## 2D Testing Stuff

In [48]:
target_size = 128
mfcc_tiled = tile_and_crop(test_X_2D['mfcc'], target_size)
chroma_tiled = tile_and_crop(test_X_2D['chroma'], target_size)
cqt_tiled = tile_and_crop(test_X_2D['cqt'], target_size)

testing_2D_melmfccchroma = np.stack((test_X_2D['melspectrogram'], mfcc_tiled, chroma_tiled), axis=-1)
testing_2D_melmfcccqt = np.stack((test_X_2D['melspectrogram'], cqt_tiled, chroma_tiled), axis=-1)

# **Evaluation**

Mel Spectrogram/MFCC/CQT Models

In [49]:
# train_data = data['train'].copy()
# val_data = data['val'].copy()
# del data

# train_labels = train_data['label'].copy()
# temp = train_data.copy()
# del temp['label']
# tr_features = temp

# val_labels = val_data['label'].copy()
# temp = val_data.copy()
# del temp['label']
# v_features = temp

# train_y, train_features = shuffle_data(train_labels, tr_features)
# training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt'], train_features['melspectrogram']), axis=1)
# training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)


# scaler = StandardScaler()
# training_features = scaler.fit_transform(training_features)

# testing_features = scaler.transform(testing_1D_melmfcccqt)
# testing_features.shape

In [50]:
# test_y_2D.shape

In [51]:
# loss, accuracy = ffnn_melmfcccqt.evaluate(testing_features, test_y_2D)

# test_results['FFNN_CQT'] = accuracy

# print(f"Test Loss: {loss}")
# print(f"Test Accuracy: {accuracy}")

In [55]:
accuracy = cnn_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['CNN_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0196 - loss: 3.1776 - weighted_accuracy: 0.0196
Test Accuracy: 0.018549051135778427


In [ ]:
accuracy = EB0_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['EB0_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

In [ ]:
accuracy = EB4_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['EB4_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

In [ ]:
accuracy = Res_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['Res_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

In [ ]:
accuracy = VGG_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['VGG_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

# **Evaluation**

Mel Spectrogram/MFCC/Chroma Models

In [ ]:
# training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['melspectrogram']), axis=1)
# training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)


# scaler = StandardScaler()
# training_features = scaler.fit_transform(training_features)

# testing_features = scaler.transform(testing_1D_melmfcccqt)
# testing_features.shape

In [ ]:
# loss, accuracy = ffnn_melmfccchroma.evaluate(testing_1D_melmfccchroma, test_y_2D)

# test_results['FFNN_Chroma'] = accuracy

# print(f"Test Loss: {loss}")
# print(f"Test Accuracy: {accuracy}")

In [ ]:
accuracy = cnn_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['CNN_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

In [ ]:
accuracy = EB0_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['EB0_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

In [ ]:
accuracy = EB4_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['EB4_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

In [ ]:
accuracy = Res_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['Res_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

In [ ]:
accuracy = VGG_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['VGG_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

## Review all the results

In [ ]:
test_results_df = pd.DataFrame(list(train_results.items()), columns=['Models', 'Testing_Accuracy']).round(3)

test_results_df = test_results_df.sort_values('Features')
test_results_df